In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import collections
from collections import defaultdict
import math
from wordcloud import WordCloud
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
from sklearn.model_selection import  cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
import fasttext
from sklearn.model_selection import cross_val_predict
import pickle
import multiprocessing
from gensim.models import Word2Vec

In [11]:
#loading the csv file

df_tweets = pd.read_csv("C:\\Users\\USER\\Documents\\Geetha\\Data Science\\Github\\Springboard\\Capstone Poject 1\\tweet_preprocessed.csv", parse_dates=True,na_values=' ',encoding="ISO-8859-1"

)


In [37]:

def join_tweets(tweet):
    tweet_tokens_joined =[' '.join(words) for words in tweet]
    return tweet_tokens_joined 

tweet_nostopwords = df_tweets['tweet_nostopwords'].tolist()
tweet_tokens_joined=join_tweets(tweet_nostopwords)
tweet_nostopwords 

["['bismillah', 'acc', 'number', 'follow', 'jzk']",
 "['assalamu', 'alaikum', 'follow']",
 "['ummlina85', 'sent', 'last', 'suspended', 'already']",
 "['rashidunwaleed', 'damn', 'probably', 'follow', 'list', 'fault', 'block']",
 "['ottomanakh', 'erdogan', 'kafir']",
 "['ottomanakh', 'cool']",
 "['ihwco', 'cool', 'blur', 'name', 'random', 'ppl', 'loockin', 'telegram', 'thx']",
 "['ihwco', 'exactly', 'hiding', 'user', 'bros', 'didn', 'screen', 'shot', 'though']",
 "['ihwco', 'thx']",
 "['bismillah', 'suspension']",
 "['guess', 'doing']",
 "['salam', 'alaykum', 'again', 'f0ll0w', 'retweet', 'post', 'jazak', 'khayr', '5upp0rt']",
 "['false']",
 "['bismillah', 'rahman', 'raheem', 'jumada', 'akhirah', '1437', 'hijri']",
 "['islam', 'caliphate', 'declared', 'sunday', '1st', 'ramadan', '1435', '29th', 'june', '2014', '643', 'passed', 'remain']",
 "['suspend', 'account', 'tos', 'violation', 'thx']",
 "['snitches']",
 "['horrifying', 'peacekeepers', 'allegedly', 'tied', 'girls', 'forced']",
 "['a

In [36]:
# using word2vec model to find similar words
#

w2v_model = Word2Vec(min_count=5,
                     window=4,
                     size=100,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20)
w2v_model.build_vocab(tweet_nostopwords, progress_per=10000)
w2v_model.train(tweet_nostopwords, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
w2v_model.init_sims(replace=True)
w2v_model.wv.most_similar(topn=10,positive='militant')

KeyError: "word 'militant' not in vocabulary"

In [13]:
# using fasttext model to find similar words

model = fasttext.train_unsupervised('tweets_clean.txt')
model.get_nearest_neighbors('militant')

[(0.8562129735946655, 'militant.'),
 (0.8422885537147522, '"militant'),
 (0.7744318246841431, 'eyeonmilitant'),
 (0.7055030465126038, 'militant"'),
 (0.6845437288284302, '155mm'),
 (0.6332287788391113, 'militants)'),
 (0.6322634220123291, 'yussuf'),
 (0.6221827268600464, 'kulgammumtaz'),
 (0.6207752823829651, 'ingushetia'),
 (0.6029382944107056, 'fatally')]

In [14]:
#Building a Naive bayes classification model using nltk library

from sklearn.model_selection import train_test_split
X=df_tweets["tweet_nostopwords"]
y=df_tweets["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=42, stratify=y)

X_train = X_train.tolist() 
X_test = X_test.tolist() 
y_train = y_train.tolist() 
y_test = y_test.tolist() 

In [15]:
def processed_tweets(X,y):
    tweets=[]
    for tweet, label in zip( X,y):
        tweets.append((tweet,label))
    return tweets

In [16]:
def buildVocabulary(train_data):
    all_words = []
    
    for (words, sentiment) in train_data:
        all_words.extend(words)
    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [17]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:        
        features['contains(%s)' % word] = (word in tweet_words)
    return features 


In [18]:
word_features = buildVocabulary(processed_tweets(X_train, y_train))
trainingFeatures = nltk.classify.apply_features(extract_features, processed_tweets(X_train, y_train))

In [19]:
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)

In [20]:
pickle.dump( NBayesClassifier , open( "tweet_pickled.p", "wb" ) )

In [21]:
NBayesClassifier = pickle.load(open( "tweet_pickled.p", "rb" ))

In [22]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in processed_tweets(X_test,y_test)]

In [23]:
#checking the model performance

def model_performance():      
    print("Accuracy:",sklearn.metrics.accuracy_score(y_test, NBResultLabels))    
    print("F1 score:",sklearn.metrics.f1_score(y_test, NBResultLabels))
    print("Precision:",sklearn.metrics.precision_score(y_test, NBResultLabels))     
    print("roc-auc score:",sklearn.metrics.roc_auc_score(y_test, NBResultLabels))
    print("Recall",sklearn.metrics.recall_score(y_test, NBResultLabels))
    print("Confusion_matrix: \n",sklearn.metrics.confusion_matrix(y_test, NBResultLabels))
model_performance()

Accuracy: 0.7096496266513498
F1 score: 0.5556043956043956
Precision: 0.7069351230425056
roc-auc score: 0.6664684345684153
Recall 0.45763939174511226
Confusion_matrix: 
 [[1839  262]
 [ 749  632]]


In [24]:
#Building a Naive bayes clssification model using scikit learn as nltk doesn't  support cross validation

def make_xy(tweet, vectorizer=None):    
    if vectorizer is None:
        vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(tweet.tweet_tokens_joined)
    X = X.tocsc()  
    y = tweet.label.values.astype(np.int)
    return X, y
X, y = make_xy(df_tweets)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [38]:
#Model performnce after cross validation

def model_performance_crossval():
    clf = MultinomialNB(alpha=.1)
    k_fold = KFold( n_splits=5, shuffle=True, random_state=0)
    print("Accuracy:", cross_val_score(clf, X, y, cv=k_fold, n_jobs=1,scoring='accuracy'))
    print("F1 score:",cross_val_score(clf, X, y, n_jobs=1, cv=k_fold,scoring='f1'))
    print("roc-auc score:",cross_val_score(clf, X, y, cv=k_fold, n_jobs=1,scoring='roc_auc'))
    print("precision:",cross_val_score(clf, X, y, cv=k_fold, n_jobs=1,scoring='precision'))
    print("recall:",cross_val_score(clf, X, y, cv=k_fold, n_jobs=1,scoring='recall'))
    y_pred = cross_val_predict(clf, X, y, cv=k_fold)
    conf_mat = confusion_matrix(y, y_pred)
    print("Confusion Matrix:\n", conf_mat)
model_performance_crossval()

Accuracy: [nan nan nan nan nan]
F1 score: [nan nan nan nan nan]
roc-auc score: [nan nan nan nan nan]
precision: [nan nan nan nan nan]
recall: [nan nan nan nan nan]


C:\IDE\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: "['bismillah', 'acc', 'number', 'follow', 'jzk']"

  FitFailedWarning)
C:\IDE\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: "['assalamu', 'alaikum', 'follow']"

  FitFailedWarning)
C:\IDE\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: "['bismillah', 'acc', 'number', 'follow', 'jzk']"

  FitFailedWarning)
C:\IDE\Anaconda\lib\site-packages\sk

ValueError: could not convert string to float: "['bismillah', 'acc', 'number', 'follow', 'jzk']"

In [39]:
#Removing empty rows and rows with a single word 
def emptyrows_remove(tweet, nostopwords):    
    min_words = 1
    tweet = tweet[nostopwords.apply(lambda x: True  if len(x)>min_words else False)]      
    return tweet

df_tweets = emptyrows_remove(df_tweets, df_tweets['tweet_nostopwords'])

In [40]:
# Checking performance after removing empty rows and rows with a single word 

X, y = make_xy(df_tweets)
model_performance_crossval()

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [41]:

df = pd.DataFrame(y,columns=['Output'])
df.to_excel(excel_writer = "C:\\Users\\USER\\Documents\\Geetha\\Data Science\\Github\\Springboard\\Capstone Poject 1\\test.xlsx",encoding="ISO-8859-1")
df_tweets['Prediction'] =df['Output']